# 모듈화 - 1

In [33]:
!pip install selenium


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:


# 모듈, 라이브러리 설치 및 셀레니움실행
# 선수기록 페이지까지 이동

# 필요 모듈, 라이브러리 가져오기
import selenium
import time
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd

# 드라이버 로드
from selenium import webdriver

# ChromeDriver 경로 설정
driver = wd.Chrome()


# K리그 데이터 포털 접속하기
target = 'https://portal.kleague.com/user/loginById.do?portalGuest=rstNE9zxjdkUC9kbUA08XQ=='
driver.get(target)

# 데이터 센터 버튼 클릭하기
driver.execute_script("javascript:moveMainFrame('0011');")

# javascript:moveMainFrame('0194'); => 부가기록 버튼 클릭하기
driver.execute_script("javascript:moveMainFrame('0194');")

# 선수별 경기기록 창 들어가기
driver.execute_script("javascript:moveMainFrame('0208');")

In [ ]:
# 모듈화 - 2 (시간 주의)
# 경기데이터 추출 및 오류 케이스 처리
# 각 데이터 데이터프레임으로 아직 저장 안됨(처리해야함)

# 페이지 내의 경기기록 테이블 추출하기
table = driver.find_element(By.CSS_SELECTOR, '#playerDataTable')
# thead
thead = table.find_element(By.TAG_NAME, "thead")
tbody = table.find_element(By.TAG_NAME, "tbody")
#break
# 표를 살펴보면 thead가 2줄로 되어 있어서 이것을 1줄로 합치는 작업을 해야한다.
thead_1 =  thead.text.strip('시도 성공 성공%').strip('\n')
# 먼저 띄어쓰기가 되어 있는 단어들을 replace를 이용하여 붙여준다.
thead_1 = thead_1.replace('유효 슈팅', '유효슈팅')
thead_1 = thead_1.replace('내 슈팅', '내슈팅')
thead_1 = thead_1.replace('외 슈팅', '외슈팅')
thead_1 = thead_1.replace('방 패스', '방패스')
thead_1 = thead_1.replace('진영 패스', '진영패스')
thead_1 = thead_1.replace('경합 지상', '지상경합')
thead_1 = thead_1.replace('경합 공중', '공중경합')


# 시도, 성공, 성공률을 특정 칼럼(원 리스트에서 붙어 있던 것들)에 결합해준다.
col = thead_1.split(' ')
column_name = []
for cl in col:
    keyword = f'{cl}시도 {cl}성공 {cl}성공%'
    if cl == '드리블':
        cl.replace(cl, keyword)
        column_name.append(keyword)
                
    elif cl == '패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)
                
    elif cl == '전방패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)

    elif cl == '후방패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)

    elif cl == '횡패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)

    elif cl == '공격진영패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)

    elif cl == '수비진영패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)

    elif cl == '중앙진영패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)                

    elif cl == '롱패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)   

    elif cl == '중거리패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)   

    elif cl == '단거리패스':
        cl.replace(cl, keyword)
        column_name.append(keyword)   

    elif cl == '크로스':
        cl.replace(cl, keyword)
        column_name.append(keyword)   

    elif cl == '지상경합':
        cl.replace(cl, keyword)
        column_name.append(keyword)   

    elif cl == '공중경합':
        cl.replace(cl, keyword)
        column_name.append(keyword)   

    elif cl == '태클':
        cl.replace(cl, keyword)
        column_name.append(keyword)   

    else:
        column_name.append(cl)
        # 문자열을 붙인 후 다시 ' ' 단위로 쪼개서 칼럼이름 리스트를 만들어줌.
cls = ' '.join(column_name)
column_name = cls.split(' ')
print('column')
time.sleep(5)
        # 빈 데이터 프레임 만들기(칼럼이름만 설정)

# 년도 선택
# 1. Select 객체 생성
year_option = '#selectYear.select-control'  # CSS 선택자
year_opt = driver.find_element(By.CSS_SELECTOR, year_option)
year_options = driver.find_elements(By.CSS_SELECTOR, '#selectYear.select-control > option')
teams_select = Select(year_opt)
year_li = year_opt.text
year_list = year_li.split('\n')
# 2. 리스트 작동 선택 (인덱스 0은 첫 번째 요소를 나타냅니다)
y_keys = [ option.text for option in year_options[1:1+4] ]  # 2023년 부터 2020년 까지의 데이터 선택
for year in y_keys:
    print('year:', year)
    year_opt = driver.find_element(By.CSS_SELECTOR, year_option)
    year_opt.send_keys( year ) 
    time.sleep(1)

    # 리그 선택
    # 년도 선택
    # 1. Select 객체 생성
    league_option = '#selectMeetSeq.select-control'  # CSS 선택자
    league_opt = driver.find_element(By.CSS_SELECTOR, league_option)
    league_options = driver.find_elements(By.CSS_SELECTOR, '#selectMeetSeq.select-control > option')
    league_select = Select(league_opt)
    league_li = league_opt.text
    league_list = league_li.split('\n')

    # 2. 리스트 작동 선택 (인덱스 0은 첫 번째 요소를 나타냅니다)
    l_keys = [ option.text for option in league_options[1:1+2] ]
    for league in l_keys:
        print('리그:', league)
        year_opt = driver.find_element(By.CSS_SELECTOR, year_option)
        league_opt = driver.find_element(By.CSS_SELECTOR, league_option)
        league_opt.send_keys( league ) 
        time.sleep(1)

        # 팀 선택
        # 1. Select 객체 생성
        team_option = '#selectTeamId.select-control'  # CSS 선택자
        team_opt = driver.find_element(By.CSS_SELECTOR, team_option)
        team_options = driver.find_elements(By.CSS_SELECTOR, '#selectTeamId.select-control > option')
        teams_select = Select(team_opt)
        team_li = team_opt.text
        team_list = team_li.split('\n')

        # 2. 리스트 작동 선택 (인덱스 0은 첫 번째 요소를 나타냅니다)
        t_keys = [ option.text for option in team_options[1:] ]
        for team in t_keys:
            #team_df=pd.DataFrame(columns=column_name)
            print('팀:', team ) 
            team_opt = driver.find_element(By.CSS_SELECTOR, '#selectTeamId')
            team_opt.send_keys( team ) 
            time.sleep(1)

            # 경기 선정하기(for 사용)
            # 1. 객체 선정
            match_option = '#selectGameId'  # CSS 선택자
            match_opt = driver.find_element(By.CSS_SELECTOR, match_option)
            match_options = driver.find_elements(By.CSS_SELECTOR, '#selectGameId.select-control > option')
            matches_select = Select(match_opt)
            search_button = '#btnSearch'
            click_button = driver.find_element(By.CSS_SELECTOR, search_button)

            # 2. 리스트 작동 선택 (인덱스 0은 첫 번째 요소를 나타냅니다)
            m_keys = [ option.text for option in match_options[1:] ]
            if len(m_keys) == 0:
                continue     # 페이지 오류로 인하여 1부 소속팀이 아닌 2부 소속팀이 먼저 나오는 경우가 있음. 이를 해결하는 조건문임.

            else:
                print('경기 수:', len(m_keys))
                # 경기 선정하기(for 사용)
                # 1. 객체 선정 
                match_option = '#selectGameId'  # CSS 선택자
                match_opt = driver.find_element(By.CSS_SELECTOR, match_option)
                match_options = driver.find_elements(By.CSS_SELECTOR, '#selectGameId.select-control > option')
                matches_select = Select(match_opt)
                search_button = '#btnSearch'
                click_button = driver.find_element(By.CSS_SELECTOR, search_button)

                # 2. 리스트 작동 선택 (인덱스 0은 첫 번째 요소를 나타냅니다)
                m_keys = [ option.text for option in match_options[1:] ]
                for match in m_keys:
                    print( '라운드 / 상대팀:', match )
                    match_opt = driver.find_element(By.CSS_SELECTOR, '#selectGameId')
                    match_opt.send_keys( match )
                    time.sleep(3)  
                    click_button = driver.find_element(By.CSS_SELECTOR, search_button)
                    click_button.click()
                    time.sleep(3)
                    # 빈 데이터 프레임 만들기(칼럼이름만 설정)
                    df=pd.DataFrame(columns=column_name)

                    datas = []
                    # 각 행을 데이터프레임에 추가하기
                    # 페이지 내의 경기기록 테이블 추출하기
                    table = driver.find_element(By.CSS_SELECTOR, '#playerDataTable')
                    # thead
                    thead = table.find_element(By.TAG_NAME, "thead")
                    tbody = table.find_element(By.TAG_NAME, "tbody")
                    table_rows = table.find_elements(By.TAG_NAME, 'tr')
                    # 빈 테이블일 경우(미진행 경기), continue를 사용하여 넘어감
                    if len(table_rows) == 3:
                        print('미진행 경기')
                        continue
                    else:
                        for table_row in table_rows[2:]:
                            row_data = table_row.text.split(' ')
                            if len(row_data) != len(column_name):
                            # 이름 열에 공백이 있는 경우
                                modified_name = "".join(row_data[1:3])  # 이름 열의 첫 번째와 두 번째 요소를 합치기
                                row_data[1:3] = [modified_name]  # 이름 열의 첫 번째와 두 번째 요소를 수정된 이름으로 대체  
                                datas.append(row_data)
                        df = pd.DataFrame(data=datas, columns=column_name)
                        print('match data \n', df) 
                        print('fin loop')
                        time.sleep(5)

                

column
year: 2023
리그: 하나원큐 K리그1
팀: 부산
팀: 전남
팀: 성남
팀: 경남
팀: 부천
팀: 안양
팀: 서울E
팀: 안산
팀: 충남아산
팀: 김천
팀: 김포
팀: 충북청주
팀: 천안
팀: 울산
경기 수: 33
라운드 / 상대팀: 1R / 전북(H)
match data 

fin loop


KeyboardInterrupt: 